In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import json
from pandas.api.types import union_categoricals
from itertools import islice
import re
import addfips
import requests
import urllib
import time
DC_STATEHOOD = 1 # Enables DC to be included in the state list
import us

c:\Users\churn_5stexhd\Utilities\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Numba needs NumPy 1.22 or greater. Got NumPy 1.21.


ImportError: cannot import name 'SearchEngine' from 'uszipcode' (c:\Users\churn_5stexhd\Utilities\anaconda3\lib\site-packages\uszipcode\__init__.py)

In [63]:
# Variable list and types we want from H-2A application data
h2a_dtype_dict = {
    'FiscalYear': 'string',
    'Matched': 'category',
    'WORKSITE_STATE': 'category',
    'WORKSITE_CITY': 'string',
    'WORKSITE_COUNTY': 'string',
    'ORIGINAL_WORKSITE_COUNTY': 'string',
    'WORKSITE_POSTAL_CODE': 'string',
    'EMPLOYER_NAME': 'string',
    'new_EMPLOYER_NAME': 'string',
    'CASE_NUMBER': 'string',
    'CASE_STATUS': 'category',
    'EMPLOYER_POSTAL_CODE': 'string',
    'JOB_TITLE': 'string',
    'NBR_WORKERS_REQUESTED': 'float32',
    'NBR_WORKERS_CERTIFIED': 'float32',
    'BASIC_NUMBER_OF_HOURS': 'float32',
    'BASIC_RATE_OF_PAY': 'float32',
    'BASIC_UNIT_OF_PAY': 'category',
    'CERTIFICATION_BEGIN_DATE': 'string',
    'CERTIFICATION_END_DATE': 'string',
    'DECISION_DATE': 'string',
    'CASE_RECEIVED_DATE': 'string',
    'ORGANIZATION_FLAG': 'category',
    'REQUESTED_START_DATE_OF_NEED': 'string',
    'REQUESTED_END_DATE_OF_NEED': 'string',
    'PRIMARY_CROP': 'string',
    'LAWFIRM_NAME': 'string',
    'SOC_CODE': 'category',
    'SOC_TITLE': 'category',
    'NAICS_CODE': 'category',
    'FULL_TIME_POSITION': 'category',
    'NATURE_OF_TEMPORARY_NEED': 'category',
    'OVERTIME_RATE_FROM': 'float32',
    'OVERTIME_RATE_TO': 'float32',
    'EDUCATION_LEVEL': 'category',
    'OTHER_EDU': 'category',
    'SWA_NAME': 'string',
    'JOB_IDNUMBER': 'string',
    'JOB_START_DATE': 'string',
    'JOB_END_DATE': 'string',
    'TRADE_NAME_DBA': 'string',
    'TYPE_OF_EMPLOYER_APPLICATION': 'category',
    'JOB_ORDER_NUMBER': 'string',
    'TOTAL_WORKERS_NEEDED': 'float32',
    'SPECIAL_REQUIREMENTS': 'string',
    'begin_year': 'string',
    'end_year': 'string',
    'case_year': 'string',
    'Abbreviation': 'category',
    'EMPLOYER_STATE': 'category',
    'EMPLOYER_CITY': 'string'
}

col_list = list(h2a_dtype_dict.keys())

In [64]:
small_rename_dict = {
    'EMPLOYMENT_BEGIN_DATE': 'JOB_START_DATE',
    'EMPLOYMENT_END_DATE': 'JOB_END_DATE',
    'REQUESTED_BEGIN_DATE': 'REQUESTED_START_DATE_OF_NEED',
    'REQUESTED_END_DATE': 'REQUESTED_END_DATE_OF_NEED',
    'TOTAL_WORKERS_H2A_REQUESTED': 'NBR_WORKERS_REQUESTED',
    'TOTAL_WORKERS_H2A_CERTIFIED': 'NBR_WORKERS_CERTIFIED'
}

small_h2a_dtype_dict = {
    'CASE_NUMBER': 'string',
    'CASE_STATUS': 'category',
    'TOTAL_WORKERS_NEEDED': 'float32',
    'TOTAL_WORKERS_H2A_REQUESTED': 'float32',
    'TOTAL_WORKERS_H2A_CERTIFIED': 'float32',
    'ANTICIPATED_NUMBER_OF_HOURS': 'float32',
    'REQUESTED_BEGIN_DATE': 'string',
    'REQUESTED_END_DATE': 'string',
    'EMPLOYMENT_BEGIN_DATE': 'string',
    'EMPLOYMENT_END_DATE': 'string',
    'WORKSITE_STATE': 'category',
    'WORKSITE_CITY': 'string',
    'WORKSITE_COUNTY': 'string',
    'WORKSITE_POSTAL_CODE': 'string'
}

small_col_list = list(small_h2a_dtype_dict.keys())

In [65]:
# # Convert from xlsx to parquet
# df = pd.read_excel("../Data/h2a/H2A Final Worksite County Data.xlsx", usecols=col_list, dtype=h2a_dtype_dict, parse_dates=False)
# df.to_parquet("../binaries/h2a_raw.parquet")

In [66]:
# Check 2021 and 2022 numbers
df_2021 = pd.read_excel("../Data/h2a/H-2A_Disclosure_Data_FY2021.xlsx", usecols=small_col_list, dtype=small_h2a_dtype_dict, parse_dates=False)
df_2021['FiscalYear'] = '2021'
df_2022 = pd.read_excel("../Data/h2a/H-2A_Disclosure_Data_FY2022_Q4.xlsx", usecols=small_col_list, dtype=small_h2a_dtype_dict, parse_dates=False)
df_2022['FiscalYear'] = '2022'

In [67]:
df_2021_2022 = pd.concat([df_2021, df_2022])
df_2021_2022 = df_2021_2022.rename(columns = small_rename_dict)
df_2021_2022['ken'] = True

H-2A dataset without Places API imputed counties

In [68]:
h2a_df = pd.read_parquet("../binaries/h2a_raw.parquet")

# Years prior to 2008 do not have worksite location information
h2a_df = h2a_df[h2a_df['FiscalYear'].astype(int) >= 2008]
h2a_df['ken'] = False

h2a_df = pd.concat([h2a_df, df_2021_2022])

# Remove non-US locations
h2a_df = h2a_df[h2a_df['WORKSITE_STATE'] != 'AB'].copy() # AB is Alberta, Canada
h2a_df = h2a_df[h2a_df['WORKSITE_STATE'] != 'SK'].copy() # SK is Saskatchewan, Canada
h2a_df = h2a_df[h2a_df['WORKSITE_STATE'] != 'MB'].copy() # MB is Manitoba, Canada

In [69]:
# Obtain list of states and counties in H-2A data
state_county_df = h2a_df[['WORKSITE_STATE', 'WORKSITE_COUNTY']].drop_duplicates()

In [70]:
# Add column with fixed names for counties
state_county_df['state'] = state_county_df['WORKSITE_STATE']
state_county_df['county'] = state_county_df['WORKSITE_COUNTY']

In [71]:
# Fix county names with multiple counties, or with city names
state_county_df.loc[state_county_df['WORKSITE_COUNTY'] == 'HARRISON & BOURBON COUNTIES', 'county'] = 'HARRISON,BOURBON'
state_county_df.loc[state_county_df['WORKSITE_COUNTY'] == 'STAMPING GROUND SCOTT COUNTY', 'county'] = 'SCOTT'
state_county_df.loc[state_county_df['WORKSITE_COUNTY'] == 'HENRY & UNION COUNTY', 'county'] = 'HENRY,UNION'
state_county_df.loc[state_county_df['WORKSITE_COUNTY'] == 'SILVER BOW AND MADISON', 'county'] = 'SILVER BOW,MADISON'
state_county_df.loc[state_county_df['WORKSITE_COUNTY'] == 'CANDLER BULLOCH & EVANS COUNTIES', 'county'] = 'CANDLER,BULLOCH,EVANS'
state_county_df.loc[state_county_df['WORKSITE_COUNTY'] == 'SAMPSON AND JOHNSTON COUNTIES', 'county'] = 'SAMPSOM,JOHNSTON'
state_county_df.loc[state_county_df['WORKSITE_COUNTY'] == 'LINCOLN AND BAYFIELD COUNTIES', 'county'] = 'LINCOLN,BAYFIELD'
state_county_df.loc[state_county_df['WORKSITE_COUNTY'] == 'ALLEN & MONROE COUNTIES', 'county'] = 'ALLEN,MONROE'
state_county_df.loc[state_county_df['WORKSITE_COUNTY'] == 'OWEN & HENRY COUNTIES', 'county'] = 'OWEN,HENRY'
state_county_df.loc[state_county_df['WORKSITE_COUNTY'] == 'UPTON/MAGNOLIA HART COUNTY', 'county'] = 'UPTON,MAGNOLIA,HART'
state_county_df.loc[state_county_df['WORKSITE_COUNTY'] == 'WARREN & SIMPSON COUNTY', 'county'] = 'WARREN,SIMPSON'
state_county_df.loc[state_county_df['WORKSITE_COUNTY'] == 'CASEY & BOYLE COUNTIES', 'county'] = 'CASEY,BOYLE'
state_county_df.loc[state_county_df['WORKSITE_COUNTY'] == 'MONTGOMERY & HALIFAX COUNTIES', 'county'] = 'MONTOGOMERY,HALIFAX'
state_county_df.loc[state_county_df['WORKSITE_COUNTY'] == 'COUNTIES: RISING SUN DILLISBORO DEARBORN', 'county'] = 'RISING SUN,DILLISBORO,DEARBORN'
state_county_df.loc[state_county_df['WORKSITE_COUNTY'] == 'GEORGE AND RANKIN COUNTIES', 'county'] = 'GEORGE,RANKIN'
state_county_df.loc[state_county_df['WORKSITE_COUNTY'] == 'MISSAUKEE WEXFORD OSCEOLA & ANTRIM COUNTIES', 'county'] = 'MISSAUKEE,WEXFORD,OSCEOLA,ANTRIM'
state_county_df.loc[state_county_df['WORKSITE_COUNTY'] == 'WASHAKIE BIG HORN JOHNSON COUNTY', 'county'] = 'WASHAKIE,BIG HORN,JOHNSON'
state_county_df.loc[state_county_df['WORKSITE_COUNTY'] == 'SOMERSET/BALD MOUNTAIN-UNORGANIZED TS', 'county'] = 'SOMERSET,BALD MOUNTAIN'
state_county_df.loc[state_county_df['WORKSITE_COUNTY'] == 'MAHLEUR AND HARNEY COUNTIES', 'county'] = 'MAHLEUR,HARNEY'
state_county_df.loc[state_county_df['WORKSITE_COUNTY'] == 'FRANKLIN & SOMERSET', 'county'] = 'FRANKLIN,SOMERSET'

In [72]:
pd.set_option('display.max_rows', 20)

In [73]:
# Create duplicate entries for applications with multiple counties that are separated with a comma
state_county_df['county_list'] = state_county_df['county'].str.split(',')
state_county_df = state_county_df.explode('county_list')

In [74]:
# Drop entries with no county or no state
state_county_df = state_county_df[(state_county_df['county_list'].notna()) & (state_county_df['state'].notna())]

In [75]:
# Use addfips package to get FIPS codes for each state-county pair
af = addfips.AddFIPS()
state_county_df['fips_nonapi'] = state_county_df.apply(lambda x: af.get_county_fips(x['county_list'], state=x['state']), axis=1)

In [76]:
# Collapse back into WORKSITE_STATE and WORKSITE_COUNTY
# Use transform as otherwise pandas freaks out when collapsed df has fewer rows than original df, producing all NaNs
state_county_df['fips_nonapi'] = state_county_df['fips_nonapi'].astype('string').fillna('')
state_county_df['fips_nonapi'] = state_county_df.groupby(['WORKSITE_STATE', 'WORKSITE_COUNTY'])['fips_nonapi'].transform(lambda x: ','.join(x))
state_county_df.drop_duplicates(subset=['WORKSITE_STATE', 'WORKSITE_COUNTY', 'fips_nonapi'], inplace=True)

In [77]:
# Add non-API county FIPS codes to entries
h2a_nonapi_df = h2a_df.merge(state_county_df, left_on=['WORKSITE_STATE', 'WORKSITE_COUNTY'], right_on=['WORKSITE_STATE', 'WORKSITE_COUNTY'], how='left')
h2a_nonapi_df = h2a_nonapi_df[h2a_nonapi_df['ken'] == False]

In [78]:
h2a_2021_2022_nonapi_df = df_2021_2022.merge(state_county_df, left_on=['WORKSITE_STATE', 'WORKSITE_COUNTY'], right_on=['WORKSITE_STATE', 'WORKSITE_COUNTY'], how='left')
# Save as binary to be processed in R
h2a_2021_2022_nonapi_df.to_parquet("../binaries/h2a_2021_2022_with_fips.parquet", index=False)

Use Places API to impute counties for unmatched entries; put in separate parquet

In [79]:
# Export list of unmatched state-county pairs
unmatched_counties_df = state_county_df[state_county_df['fips_nonapi'] == '']

unmatched_counties_df = unmatched_counties_df.assign(place_name = unmatched_counties_df['county_list'] + ', ' + unmatched_counties_df['state'].astype(str))
place_names_list = list(unmatched_counties_df['place_name'])

In [80]:
# Use Google Places API to fill in missing county names if possible
# Google maps API key from my account
# Import API key stored in text file
with open("../tools/google_places_api_key.txt") as f:
    lines = f.readlines()

api_key = lines[0]

In [81]:
# # Make API call for each place name to obtain Google Places placeID
# Base url to call findplace API
# base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?"
# placeid_dict = {}
# for place_ind in range(0, len(place_names_list)):
    
#     place_name = place_names_list[place_ind]
#     print('Currently looking up ' + place_name)

#     # Create API request
#     # URL'ed location name we want to search
#     input = urllib.parse.quote(place_name) # Encode place name as URL string
#     request_url = base_url + "input=" + input + "&inputtype=textquery" + "&key=" + api_key

#     payload = {}
#     headers = {}

#     # Sleep one second between each API call
#     time.sleep(1)

#     # Make API call
#     response = requests.request("GET", request_url, headers=headers, data=payload)
#     response_json = response.json()
    
#     # If API call is successful, then place response result into dict
#     if response_json['status']=='OK':
#         print('Successful')
#         placeid_dict[place_name] = response_json
#     else:
#         # If API call is unsuccessful, then wait 5 seconds and retry
#         print('NOT successful, retrying')
#         time.sleep(5)
#         response = requests.request("GET", request_url, headers=headers, data=payload)
#         response_json = response.json()

#         if response_json['status']=='OK':
#             print('Retry successful')
#             placeid_dict[place_name] = response_json
#         else:
#             error_type = response_json['status']
#             print('Retry unsuccessful, error: ' + error_type)

In [82]:
# # Save API request results as JSON
# with open('json/placeid_api_request_result.json', 'w') as f:
#     json.dump(placeid_dict, f)

In [83]:
# Load JSON of API responses and put into DataFrame
with open('json/placeid_api_request_result.json', 'r') as infile:
    placeid_dict = json.load(infile)

In [84]:
# Put place IDs into DataFrame
placeid_df = pd.DataFrame(columns=['name_to_search', 'placeid'])

for search_term, response in placeid_dict.items():
    number_of_candidates = len(response['candidates'])
    for response_ind in range(0, number_of_candidates):
        address = response['candidates'][response_ind]['place_id']
        placeid_df.loc[len(placeid_df)] = [search_term, address]

In [85]:
# # Use Place details API to get county names
# # base_url = 'https://maps.googleapis.com/maps/api/place/details/json?'
# place_details_dict = {}

# # Iterate over each place ID
# for index, row in placeid_df.iterrows():
#     print(row['name_to_search'], row['placeid'])

#     # Create API request
#     input = row['placeid']
#     place_name = row['name_to_search']
#     request_url = base_url + "placeid=" + input + "&key=" + api_key

#     payload = {}
#     headers = {}

#     response = requests.request("GET", request_url, headers=headers, data=payload)
#     response_json = response.json()

#     # If API call is successful, then place response result into dict
#     if response_json['status']=='OK':
#         print('Successful')
#         place_details_dict[input] = response_json
#     else:
#         # If API call is unsuccessful, then wait 5 seconds and retry
#         print('NOT successful, retrying')
#         time.sleep(5)
#         response = requests.request("GET", request_url, headers=headers, data=payload)
#         response_json = response.json()

#         if response_json['status']=='OK':
#             print('Retry successful')
#             place_details_dict[input] = response_json
#         else:
#             error_type = response_json['status']
#             print('Retry unsuccessful, error: ' + error_type)

In [86]:
# # Save API request results as JSON
# with open('json/place_details_api_request_result.json', 'w') as f:
#     json.dump(place_details_dict, f)

In [87]:
# Load JSON of API responses and put into DataFrame
with open('json/place_details_api_request_result.json', 'r') as infile:
    place_details_dict = json.load(infile)

In [88]:
# Store county name from place details into dictionary (store state names too as there may be incorrect states)
county_name_dict = {}
state_name_dict = {}

for placeid, place_ind in place_details_dict.items():
    address_components_dict_list = place_details_dict[placeid]['result']['address_components']
    for component_ind in range(len(address_components_dict_list)):
        # Address components administrative level
        component_admin_level = address_components_dict_list[component_ind]['types'][0]
        
        # County name
        if component_admin_level == 'administrative_area_level_2':
            county_name_dict[placeid] = address_components_dict_list[component_ind]['long_name']
        # State name
        elif component_admin_level == 'administrative_area_level_1':
            state_name_dict[placeid] = address_components_dict_list[component_ind]['long_name']

In [89]:
# Convert state and county name dicts into DataFrames
placeid_state_df = pd.DataFrame.from_dict(state_name_dict, orient='index', columns=['state'])
placeid_state_df.reset_index(inplace=True)
placeid_state_df.rename(columns={'index': 'placeid'}, inplace=True)

placeid_county_df = pd.DataFrame.from_dict(county_name_dict, orient='index', columns=['county'])
placeid_county_df.reset_index(inplace=True)
placeid_county_df.rename(columns={'index':'placeid'}, inplace=True)

In [90]:
# Add state and county names back into placeid_df
placeid_df = placeid_df.merge(placeid_state_df, left_on='placeid', right_on='placeid', how='left')
placeid_df = placeid_df.merge(placeid_county_df, left_on='placeid', right_on='placeid', how='left')

In [91]:
# Fix a few incorrect entries
placeid_df.loc[placeid_df['name_to_search'] == 'WASHINGOTN, ME', 'county'] = 'Washington County'
placeid_df.loc[placeid_df['name_to_search'] == 'FREDRICK, VA', 'county'] = 'Frederick County'
placeid_df.loc[placeid_df['name_to_search'] == 'UPTON, KY', 'county'] = 'Hardin County' # Upton is mostly located in Hardin county
placeid_df.loc[placeid_df['name_to_search'] == 'CA, CA', 'county'] = np.nan
placeid_df.loc[placeid_df['name_to_search'] == 'MOFFAT, WY', 'county'] = 'Moffat County'
placeid_df.loc[placeid_df['name_to_search'] == 'MOFFAT, WY', 'state'] = 'CO' # Moffat County is on the WY/CO border in CO
placeid_df.loc[placeid_df['name_to_search'] == 'MADERA, ID', 'county'] = np.nan # Not sure what Madera, ID is
placeid_df.loc[placeid_df['name_to_search'] == 'MADERA, ID', 'state'] = np.nan
placeid_df.loc[placeid_df['name_to_search'] == 'GOODING, CO', 'county'] = 'Boulder County' # Gooding is in Boulder County, CO
placeid_df.loc[placeid_df['name_to_search'] == 'GOODING, CO', 'state'] = 'CO'
placeid_df.loc[placeid_df['name_to_search'] == 'CUSTER, ND', 'county'] = np.nan # Not sure what CUSTER, ND refers to
placeid_df.loc[placeid_df['name_to_search'] == 'CUSTER, ND', 'state'] = np.nan
placeid_df.loc[placeid_df['name_to_search'] == 'ORANGE, OK', 'county'] = np.nan # Not sure what ORANGE, OK refers to
placeid_df.loc[placeid_df['name_to_search'] == 'ORANGE, OK', 'state'] = np.nan
placeid_df.loc[placeid_df['name_to_search'] == 'ALACHUA, GA', 'county'] = 'Alachua County' # Alachua County is in Florida
placeid_df.loc[placeid_df['name_to_search'] == 'ALACHUA, GA', 'state'] = 'Florida'
placeid_df.loc[placeid_df['name_to_search'] == 'MOFFAT, FL', 'county'] = np.nan # There is no place called Moffat in Florida
placeid_df.loc[placeid_df['name_to_search'] == 'MOFFAT, FL', 'state'] = np.nan
placeid_df.loc[placeid_df['name_to_search'] == 'JONES, SC', 'county'] = np.nan # There is no Jones in SC
placeid_df.loc[placeid_df['name_to_search'] == 'JONES, SC', 'state'] = np.nan
placeid_df.loc[placeid_df['name_to_search'] == 'DUNN, SD', 'county'] = np.nan # There is no Dunn in SD
placeid_df.loc[placeid_df['name_to_search'] == 'DUNN, SD', 'state'] = np.nan
placeid_df.loc[placeid_df['name_to_search'] == 'DICKEY, SD', 'county'] = np.nan # There is no Dickey in SD
placeid_df.loc[placeid_df['name_to_search'] == 'DICKEY, SD', 'state'] = np.nan

# Western Connecticut consists of Lichtfield and Fairfield counties
connecticut_row = placeid_df[placeid_df['name_to_search'] == 'WESTERN CONNECTICUT, CT'].copy()
connecticut_row['county'] = 'Litchfield County'
placeid_df = pd.concat([placeid_df, connecticut_row], ignore_index=True)

In [92]:
# Drop locations that do not have a county name (including those incorrectly resolved that I removed)
placeid_df.dropna(subset=['county'], inplace=True)

In [93]:
# Add FIPS code
placeid_df['fips'] = placeid_df.apply(lambda x: af.get_county_fips(x['county'], state=x['state']), axis=1)

In [94]:
# LaSalle Parish in Louisiana is not in the addfips package(?), so add it manually
placeid_df.loc[placeid_df['name_to_search'] == 'LASALLE, LA', 'fips'] = '22059'

In [95]:
# Keep only place names and FIPS codes, then collapse to unique place names
placeid_df = placeid_df[['name_to_search', 'fips']]
placeid_df = placeid_df.groupby(['name_to_search']).agg({'fips': lambda x: x.tolist()}).reset_index()

In [96]:
# Prepare to append back into original DataFrame of states and counties
placeid_df.rename(columns={'name_to_search': 'place_name'}, inplace=True)
unmatched_counties_df.drop(columns=['fips_nonapi'], inplace=True)
api_counties_df = unmatched_counties_df.merge(placeid_df, left_on='place_name', right_on='place_name', how='left')
api_counties_df.drop(columns=['place_name'], inplace=True)

In [97]:
# Do some final cleaning
api_counties_df.loc[api_counties_df['WORKSITE_COUNTY'] == 'KNOK', 'fips'] = '23027' # Knox town in Maine was misspelled as KNOK
api_counties_df.loc[api_counties_df['WORKSITE_COUNTY'] == 'VARIOUS COUNTIES IN WESTERN', 'fips'] = np.nan # Various counties in NC not specified
api_counties_df.loc[api_counties_df['WORKSITE_COUNTY'] == 'VARIOUS COUNTIES', 'fips'] = np.nan # Various counties in NC not specified

In [98]:
# Convert multi-FIPS entries from list to string
api_counties_df = api_counties_df[api_counties_df['fips'].notna()]
api_counties_df['fips_string'] = api_counties_df['fips'].apply(lambda x: ','.join(map(str, x)))

In [99]:
api_counties_df.loc[api_counties_df['WORKSITE_COUNTY'] == 'KNOK', 'fips_string'] = '23027' # Somehow FIPS cannot be stored as a list, so join splits the string; this fixes it

In [100]:
# We want to merge when all FIPS codes are stored as strings only
api_counties_df.drop(columns=['fips'], inplace=True)
api_counties_df['fips_string'] = api_counties_df['fips_string'].astype('string').fillna('')

In [101]:
# Combine all FIPS codes for each WORKSITE_STATE and WORKSITE_COUNTY
merged_df = state_county_df.merge(api_counties_df, left_on=['WORKSITE_STATE', 'WORKSITE_COUNTY', 'state', 'county', 'county_list'], right_on=['WORKSITE_STATE', 'WORKSITE_COUNTY',  'state', 'county', 'county_list'], how='left')

In [102]:
# Combine FIPS codes from both sources
merged_df['fips_api'] = merged_df['fips_nonapi'].where(merged_df['fips_string'].isna(), merged_df['fips_string'])

In [103]:
# Collapse back into unique WORKSITE_STATE and WORKSITE_COUNTY pairs
merged_df = merged_df[['WORKSITE_STATE', 'WORKSITE_COUNTY', 'fips_api']]
merged_df.loc[:, 'fips_api'] = merged_df['fips_api'].astype(str)

In [104]:
# Use transform as otherwise pandas freaks out when collapsed df has fewer rows than original df, producing all NaNs
merged_df['fips_api'] = merged_df.groupby(['WORKSITE_STATE', 'WORKSITE_COUNTY'])['fips_api'].transform(lambda x: ','.join(x))
merged_df.drop_duplicates(subset=['WORKSITE_STATE', 'WORKSITE_COUNTY', 'fips_api'], inplace=True)

In [105]:
# Merge back into H-2A data
h2a_merged_df = h2a_nonapi_df.merge(merged_df, left_on=['WORKSITE_STATE', 'WORKSITE_COUNTY'], right_on=['WORKSITE_STATE', 'WORKSITE_COUNTY'], how='left')

In [106]:
# For the remaining entries, we will try to get counties from the WORKSITE_CITY names
state_city_df = h2a_merged_df[h2a_merged_df['fips_api'].isna()][['WORKSITE_STATE', 'WORKSITE_CITY']].dropna().drop_duplicates().reset_index(drop=True)

In [107]:
# United States of America Python Dictionary to translate States,
# Districts & Territories to Two-Letter codes and vice versa.
#
# Canonical URL: https://gist.github.com/rogerallen/1583593
#
# Dedicated to the public domain.  To the extent possible under law,
# Roger Allen has waived all copyright and related or neighboring
# rights to this code.  Data originally from Wikipedia at the url:
# https://en.wikipedia.org/wiki/ISO_3166-2:US
#
# Automatically Generated 2021-09-11 18:04:36 via Jupyter Notebook from
# https://gist.github.com/rogerallen/d75440e8e5ea4762374dfd5c1ddf84e0 

us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI"
}
# invert the dictionary
abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))

In [108]:
# Use state name instead of abbreviation; suspect that Google Places API is more accurate with state names
state_city_df['state_name'] = state_city_df['WORKSITE_STATE'].map(abbrev_to_us_state)
state_city_df['city_state_name'] = state_city_df['WORKSITE_CITY'] + ', ' + state_city_df['state_name']

In [109]:
# # Split API search into chunks of 100
# # Base url to call findplace API
# base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?"

# for x in range(1, 17):
#     x1 = x*100
#     x2 = (x+1)*100
#     df = state_city_df.iloc[x1:x2]

#     # Make API call for each place name to obtain Google Places placeID
#     city_placeid_dict = {}

#     for ind, row in df.iterrows():
#         place_name = row['city_state_name']
#         print('Currently looking up ' + place_name)

#         # Create API request
#         # URL'ed location name we want to search
#         input = urllib.parse.quote(place_name) # Encode place name as URL string
#         request_url = base_url + "input=" + input + "&inputtype=textquery" + "&key=" + api_key

#         payload = {}
#         headers = {}

#         # Sleep one second between each API call
#         time.sleep(1)

#         # Make API call
#         response = requests.request("GET", request_url, headers=headers, data=payload)
#         response_json = response.json()
        
#         # If API call is successful, then place response result into dict
#         if response_json['status']=='OK':
#             print('Successful')
#             city_placeid_dict[place_name] = response_json
#         else:
#             # If API call is unsuccessful, then wait 5 seconds and retry
#             print('NOT successful, retrying')
#             time.sleep(5)
#             response = requests.request("GET", request_url, headers=headers, data=payload)
#             response_json = response.json()

#             if response_json['status']=='OK':
#                 print('Retry successful')
#                 city_placeid_dict[place_name] = response_json
#             else:
#                 error_type = response_json['status']
#                 print('Retry unsuccessful, error: ' + error_type)

#     # Save API request results as JSON
#     with open(f'json/city_placeid_api_request_result_{x}.json', 'w') as f:
#         json.dump(city_placeid_dict, f)

In [110]:
# Load JSON of API responses containing place IDs and put into dictionary
city_placeid_dict = {}
for x in range(0, 17):
    with open(f'json/city_placeid_api_request_result_{x}.json', 'r') as infile:
        city_placeid_dict[x] = json.load(infile)

In [111]:
# Put place IDs into DataFrame
placeid_df = pd.DataFrame(columns=['city_state_name', 'placeid'])

for x in range(0, 17):
    for search_term, response in city_placeid_dict[x].items():
        number_of_candidates = len(response['candidates'])
        for response_ind in range(0, number_of_candidates):
            address = response['candidates'][response_ind]['place_id']
            placeid_df.loc[len(placeid_df)] = [search_term, address]

In [112]:
# Keep only results with no uncertainty in location
placeid_df_no_duplicates = placeid_df.drop_duplicates(subset=['city_state_name'], keep=False)

In [113]:
# # Now split up API requests into chunks of 100
# placeid_df_no_duplicates.reset_index(inplace=True, drop=True)
# # Base url to call Place details API
# base_url = 'https://maps.googleapis.com/maps/api/place/details/json?'

# for x in range(1, 16):
#     x1 = x*100
#     x2 = (x+1)*100
#     df = placeid_df_no_duplicates.iloc[x1:x2]

#     # Use Place details API to get county names
#     place_details_dict = {}

#     # Iterate over each place ID
#     for index, row in df.iterrows():
#         print(row['city_state_name'], row['placeid'])

#         # Create API request
#         input = row['placeid']
#         place_name = row['city_state_name']
#         request_url = base_url + "placeid=" + input + "&key=" + api_key

#         payload = {}
#         headers = {}

#         response = requests.request("GET", request_url, headers=headers, data=payload)
#         response_json = response.json()

#         # If API call is successful, then place response result into dict
#         if response_json['status']=='OK':
#             print('Successful')
#             place_details_dict[input] = response_json
#         else:
#             # If API call is unsuccessful, then wait 5 seconds and retry
#             print('NOT successful, retrying')
#             time.sleep(5)
#             response = requests.request("GET", request_url, headers=headers, data=payload)
#             response_json = response.json()

#             if response_json['status']=='OK':
#                 print('Retry successful')
#                 place_details_dict[input] = response_json
#             else:
#                 error_type = response_json['status']
#                 print('Retry unsuccessful, error: ' + error_type)

#     # Save API request results as JSON
#     with open(f'json/city_place_details_api_request_result_{x}.json', 'w') as f:
#         json.dump(place_details_dict, f)

In [114]:
# Store county name from place details into dictionary (store state names too as there may be incorrect states)
county_name_dict = {}
state_name_dict = {}

for x in range(0, 16):
    # Load JSON of API responses and put into DataFrame
    with open(f'json/city_place_details_api_request_result_{x}.json', 'r') as infile:
        place_details_dict = json.load(infile)

    for placeid, place_ind in place_details_dict.items():
        address_components_dict_list = place_details_dict[placeid]['result']['address_components']
        for component_ind in range(len(address_components_dict_list)):
            # Address components administrative level
            component_admin_level = address_components_dict_list[component_ind]['types'][0]
            
            # County name
            if component_admin_level == 'administrative_area_level_2':
                county_name_dict[placeid] = address_components_dict_list[component_ind]['long_name']
            # State name
            elif component_admin_level == 'administrative_area_level_1':
                state_name_dict[placeid] = address_components_dict_list[component_ind]['long_name']

In [115]:
# Convert state and county name dicts into DataFrames
placeid_state_df = pd.DataFrame.from_dict(state_name_dict, orient='index', columns=['state'])
placeid_state_df.reset_index(inplace=True)
placeid_state_df.rename(columns={'index': 'placeid'}, inplace=True)

placeid_county_df = pd.DataFrame.from_dict(county_name_dict, orient='index', columns=['county'])
placeid_county_df.reset_index(inplace=True)
placeid_county_df.rename(columns={'index':'placeid'}, inplace=True)

In [116]:
# Add state and county names back into placeid_df
placeid_df_no_duplicates = placeid_df_no_duplicates.merge(placeid_state_df, left_on='placeid', right_on='placeid', how='left')
placeid_df_no_duplicates = placeid_df_no_duplicates.merge(placeid_county_df, left_on='placeid', right_on='placeid', how='left')

In [117]:
# Fix a few incorrect entries
placeid_df_no_duplicates.loc[placeid_df_no_duplicates['city_state_name'] == 'SEE BELOW, New York', 'county'] = np.nan

# Drop locations that do not have a county name (including those incorrectly resolved that I removed)
placeid_df_no_duplicates.dropna(subset=['state', 'county'], inplace=True)

In [118]:
# Add FIPS code
placeid_df_no_duplicates['fips_api_city'] = placeid_df_no_duplicates.apply(lambda x: af.get_county_fips(x['county'], state=x['state']), axis=1)

In [119]:
# Merge FIPS code back to original DataFrame
placeid_df_no_duplicates.loc[:, 'fips_api_city'] = placeid_df_no_duplicates['fips_api_city'].astype('string')
placeid_df_no_duplicates = placeid_df_no_duplicates[['city_state_name', 'fips_api_city']]
state_city_df = state_city_df.merge(placeid_df_no_duplicates, left_on='city_state_name', right_on='city_state_name', how='left')
state_city_df.drop(columns=['state_name', 'city_state_name'], inplace=True)

h2a_merged_df = h2a_merged_df.merge(state_city_df, left_on=['WORKSITE_STATE', 'WORKSITE_CITY'], right_on=['WORKSITE_STATE', 'WORKSITE_CITY'], how='left')

In [121]:
# Create finalized combined FIPS column
h2a_merged_df['fips_api'] = h2a_merged_df['fips_api'].astype('string').fillna('')
h2a_merged_df['fips_api_city'] = h2a_merged_df['fips_api_city'].astype('string').fillna('')
h2a_merged_df.loc[h2a_merged_df['fips_api']=='', 'fips_api'] = h2a_merged_df['fips_api_city']

In [122]:
# Save as binary to be processed in R
h2a_merged_df.to_parquet("../binaries/h2a_with_fips.parquet", index=False)